In [2]:
import numpy
G = DihedralGroup(9)
exponent = G.exponent()
o = G.order()
print("The exponent of the group is " + exponent.str())
# find the correct prime p for which we can translate the problem into Z/pZ for
def find_dixon_prime(exponent,G):
    candidate = next_prime(ceil(2*sqrt(G.order())))
    while (candidate % exponent != 1):
        candidate = next_prime(candidate+1)
    return candidate
    
p = find_dixon_prime(exponent,G)
print("the prime p is " + p.str())
F = FiniteField(p) # will work over field F = Z/pZ

# also for the conversion to finite fields require some z with ord_p(z) = e
# inefficient algorithm below (but is asymptotically irrelevant)
for candidate in range(1,p):
    if F(candidate).multiplicative_order() == exponent:
        z = F(candidate)
QZ = CyclotomicField(exponent)

def find_common_eigenvectors(matrices): #this is O(k^6)
    """
    Find k common eigenvectors for k matrices over a finite field.
    
    Parameters:
    -----------
    matrices : list of k square matrices of the same dimension over a finite field
    
    Returns:
    --------
    list of k common eigenvectors
    """
    # sanity checks
    if not matrices:
        raise ValueError("Empty list of matrices")
    
    k = matrices[0].nrows()
    F = matrices[0].base_ring()
    
    # verify all matrices are the same size and over the same field
    if any(m.nrows() != k or m.base_ring() != F for m in matrices[1:]):
        raise ValueError("Matrices must be same size and over same field")
    
    # start with eigenvectors of the first matrix
    common_eigenspaces = matrices[0].eigenspaces_right()
    
    for matrix in matrices[1:]:
        new_common_eigenspaces = []
        
        # check each current eigenspace against the next matrix
        for (eigenvalue1, eigenspace1) in common_eigenspaces:
            current_matrix_eigenspaces = matrix.eigenspaces_right()
            
            # look for matching eigenspaces
            for (eigenvalue2, eigenspace2) in current_matrix_eigenspaces:
                # check if eigenspaces intersect
                intersection = eigenspace1.intersection(eigenspace2) # this is gaussian elimination, O(k^3)
                
                # if non-zero intersection, keep it
                if intersection.dimension() > 0:
                    new_common_eigenspaces.append((eigenvalue1, intersection))
        common_eigenspaces = new_common_eigenspaces
        
        # If no common eigenspaces remain, raise an error
        if not common_eigenspaces:
            raise ValueError("No common eigenvectors found")
    
    # return basis vectors for the final common eigenspaces
    # ensure we return exactly k linearly independent vectors
    common_vectors = []
    for (_, eigenspace) in common_eigenspaces:
        # extend our list of vectors while maintaining linear independence
        for v in eigenspace.basis():
            # try to add vector while maintaining linear independence
            test_space = (F^k).span(common_vectors + [v])
            if test_space.dimension() > len(common_vectors):
                common_vectors.append(v)
                
                # stop if we have k vectors
                if len(common_vectors) == k:
                    return common_vectors
    
    # if we couldn't find k linearly independent vectors
    raise ValueError(f"Could not find {k} linearly independent common eigenvectors")

The exponent of the group is 18
the prime p is 19


In [3]:
# refer to Dixon's 1967 High Speed Computation of Group Characters paper for notation and implementation details

class DixonCharacterTable:
    def structure_constants(self,G):
        # method computing the structure constants of a group, i.e. a_rst is the number of pairs (x,y) in the conjugacy classes r and s such
        # that their product is in the conjugacy class t.

        # initialise empty dictionary
        result = {(i,j):([0]*self.classes_amount) for i in range(0,self.classes_amount) for j in range(0,self.classes_amount)}

        #easy to see following has 5 for-loops and thus runtime O(n^2*k)
        for i in range(0,self.classes_amount):
            for g in self.classes[i]:
                for j in range(0,self.classes_amount):
                    for h in self.classes[j]:
                        for k in range(0,self.classes_amount):
                            if (g*h == self.representatives[k]):
                                result[(i,j)][k] += 1
        return result
        
    def __init__(self, G):
        # initialise important data that will be used often during the computations

        # create a list of all conjugacy classes and their amount k, and fix a representative from each class
        self.classes = list(G.conjugacy_classes())
        self.classes_amount = len(self.classes)
        self.representatives = [k.representative() for k in self.classes]
        
        # compute the sizes of their classes, then in the notation of Dixon (1967) have h_j = class_sizes[j]
        self.class_sizes = [G.order() / G.centralizer(C.representative()).order() for C in self.classes]

        # compute the inverse class for each class, in Dixon (1967) have j' = inverse_class[j]
        # this is O(n^2) if we can hash conjugacy classes, but this isn't possible in sage
        # so it is O(nk^2)
        self.inverse_class = []
        for class_id in range(0,self.classes_amount):
            rep = self.representatives[class_id]
            inverse = rep.inverse()
            # compute which class contains the inverse
            for idx, cls in enumerate(self.classes,1):
                if inverse in cls:
                    self.inverse_class.append(idx-1)
                    break
        # compute the power classes for each class, for the powers 0,...,e-1. Required for conversion from finite field
        self.power_class = [ [] for _ in range(self.classes_amount)]
        for class_id in range(0, self.classes_amount):
            rep = self.representatives[class_id]
            self.power_class[class_id] = [];
            for power in range(0, exponent):
                element_to_check = rep^power
                # compute which class contains the power
                for idx, cls in enumerate(self.classes,1):
                    if element_to_check in cls:
                        self.power_class[class_id].append(idx-1)
                        break
        # initialise class matrices M_1, ..., M_k
        # computing all structure constants can be done in O(n^3)
        self.structure_constants = self.structure_constants(G)
        self.class_matrices = [matrix(F, self.classes_amount, self.classes_amount, lambda x,y: self.structure_constants[(i,x)][y]) for i in range(0,self.classes_amount)]
    def compute_normalised_vectors(self):
         v = find_common_eigenvectors(self.class_matrices)
         for idx in range(0, len(v)):
             # invert with first element
             inverse = F.one()/(v[idx][0])
             v[idx] = inverse*v[idx]
         self.normalised_vecs = v
    # run after running compute_normalised_vecs
    def compute_dimensions(self):
        # runtime O(k^2)
        result = []
        for i in range(0, self.classes_amount):
            #k_i = |G|/d_i^2
            k = sum([(self.normalised_vecs[i][j]*self.normalised_vecs[i][self.inverse_class[j]])/F(self.class_sizes[j]) for j in range(0, self.classes_amount)])
            d_squared = o*F.one()/k
            # pick a square root of d_squared by Euler's criterion
            d = d_squared.sqrt(p)
            # we know that p > 2d, so
            if (p < 2*d):
                result.append(d)
            else:
                result.append(p-d)
        self.dimensions = result
    def compute_characters(self):
        # set the character table containing the finite field elements
        self.finite_field_chartable = matrix(F, self.classes_amount, self.classes_amount, lambda x,y: self.normalised_vecs[x][y]*self.dimensions[x]/F(self.class_sizes[y]))
        # convert them to complex numbers in Z[\zeta_e] with the formula from Dixon (1967)
        # implement the finite field to complex number conversion here
        zeta = QZ.gen()
        def convert_to_C(i,j):
            # converts theta(\chi_j^i), returns \chi_j^i
            # has runtime O(exponent^2)
            result = 0
            for s in range(0,exponent):
            # compute m(s) and add it to result
                m = 0
                for n in range(0,exponent):
                    m += self.finite_field_chartable[i,self.power_class[j][n]]*z^(-s*n)
                result += int(m/exponent) * zeta^s
            return result

        # the following runs convert_to_C n^2 times, so in total have runtime O(n^2*exponent^2)
        self.chartable = matrix(QZ, self.classes_amount, self.classes_amount, lambda x,y: convert_to_C(x,y))

    def chartable_detailed(self):
        print("The class matrices of G are ")
        print(D.class_matrices)
        print("The conjugacy class sizes of G are ")
        print(D.class_sizes)
        D.compute_normalised_vectors()
        D.compute_dimensions()
        print("The dimensions of the irreps of G are ")
        print(D.dimensions)
        D.compute_characters()
        print("The chartable in the finite field Z/pZ is ")
        print(D.finite_field_chartable)
        print("The chartable in the cyclotomic extension Q(zeta_" + exponent.str() + ") is ")
        print(D.chartable)
    def chartable(self):
        # worst case runtime O(k^6+k^2n^2)
        D.compute_normalised_vectors() # O(k^6)
        D.compute_dimensions() #O(k^2)
        D.compute_characters() #O(k^2*exponent^2)
        print(D.chartable)
                            

In [4]:
def simplify_character_table(ct):
    from sage.all import QQbar
    
    def try_simplify(entry):
        try:
            # Convert to algebraic number
            alg = QQbar(entry)
            # Check if it's real
            if alg.imag().is_zero():
                return alg.real()
            return entry
        except:
            return entry
            
    # Apply to each entry in the character table
    return [[try_simplify(entry) for entry in row] for row in ct]

In [6]:
D = DixonCharacterTable(G)
D.chartable()
#print(simplify_character_table(G.character_table()))

[                            1                            -1                             1                             1                             1                             1]
[                            1                             1                             1                             1                             1                             1]
[                            2                             0                            -1                            -1                             2                            -1]
[                            2                             0  zeta18^5 - zeta18^2 - zeta18 -zeta18^4 + zeta18^2 + zeta18                            -1          -zeta18^5 + zeta18^4]
[                            2                             0          -zeta18^5 + zeta18^4  zeta18^5 - zeta18^2 - zeta18                            -1 -zeta18^4 + zeta18^2 + zeta18]
[                            2                             0 -zeta18^4 + zeta18^2 + zeta18